## AuxTel Image Header Check

In [ ]:
import logging
import re
import sys

from astropy.time import Time, TimeDelta
import numpy as np

import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient
import lsst.sitcom.mareuter.header_checks as hc
import lsst.sitcom.mareuter.header_utils as hu
import lsst.sitcom.mareuter.site_efd as se

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
efd_name = se.get_efd()
collections = ["LATISS/raw/all"]
instrument = "LATISS"
butler = dafButler.Butler("/repo/embargo", collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [ ]:
obs_date = 20230321
seq_num = 53
records = butler.registry.queryDimensionRecords("exposure", where=" and ".join([f"exposure.day_obs={obs_date}", f"exposure.seq_num={seq_num}"]))
dataId = {"instrument": instrument, "detector": 0, "exposure.id": list(records)[0].id}
raw = butler.get('raw.metadata', dataId)

In [ ]:
header = raw.toDict()
#print(header)

In [ ]:
delta = TimeDelta(120.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai").utc
obs_end = Time(header["DATE-END"], format="fits", scale="tai").utc
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
obs_id = header["OBSID"]
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")
print(f"OBSID: {obs_id}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.ATCamera.logevent_startIntegration", "*",
                                               obs_beg, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.ATCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
date_obs = Time(hu.get_value(end_image_tel, "timestampDateObs"), format="unix_tai", scale="tai")
date_end = Time(hu.get_value(end_image_tel, "timestampDateEnd"), format="unix_tai", scale="tai")
date_start_int = Time(hu.get_value(start_int, "private_sndStamp"), format="unix_tai", scale="tai").utc
date_eoit = Time(hu.get_value(end_image_tel, "private_sndStamp"), format="unix_tai", scale="tai").utc
print(date_start_int.isot)
print(date_eoit.isot)

In [ ]:
hc.check("DATE-BEG", header, date_obs.isot)
hc.check("DATE-END", header, date_end.isot)

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
image_readout_params_df = await client.select_top_n("lsst.sal.ATCamera.logevent_imageReadoutParameters",
                                                    "*", 1,
                                                    time_cut=date_start_int.isot)

In [ ]:
hc.check("OBSID", header, hu.get_value(start_int, "imageName"))
hc.check("CAMCODE", header, hu.get_value(start_int, "imageSource"))
hc.check("CONTRLLR", header, hu.get_value(start_int, "imageController"))
hc.check("DAYOBS", header, hu.get_value(start_int, "imageDate"))
hc.check("SEQNUM", header, hu.get_value(start_int, "imageNumber"))
take_image_kv = hu.dict_from_additional(start_int)
hc.check_take_image("GROUPID", header, take_image_kv, "groupId")
hc.check_take_image("IMGTYPE", header, take_image_kv, "imageType")
hc.check_take_image("TESTTYPE", header, take_image_kv, "testType")
hc.check_take_image("PROGRAM", header, take_image_kv, "program")
hc.check_take_image("REASON", header, take_image_kv, "reason")
hc.check("OBSANNOT", header, hu.get_value(image_readout_params_df, "daqAnnotation"))
hc.check("CURINDEX", header, hu.get_value(start_int, "imageIndex"))
hc.check("MAXINDEX", header, hu.get_value(start_int, "imagesInSequence"))
hc.check_not_empty("DETSIZE", header)
hc.check("OVERH", header, hu.get_value(image_readout_params_df, "overCols"))
hc.check("OVERV", header, hu.get_value(image_readout_params_df, "overRows"))
hc.check("PREH", header, hu.get_value(image_readout_params_df, "preCols"))
hc.check_float("EXPTIME", header, hu.get_value(start_int, "exposureTime"))
hc.check_float("DARKTIME", header, hu.get_value(end_image_tel, "darkTime"), precision=1e-5)
hc.check_float("SHUTTIME", header, hu.get_value(end_image_tel, "measuredShutterOpenTime"))

In [ ]:
current_target_df = await client.select_top_n("lsst.sal.ATPtg.logevent_currentTarget", "*", 1,
                                              time_cut=date_start_int.isot)
current_target_status_start_df = await client.select_top_n("lsst.sal.ATPtg.currentTargetStatus", "*", 1,
                                                           time_cut=date_start_int.isot)
current_target_status_end_df = await client.select_top_n("lsst.sal.ATPtg.currentTargetStatus", "*", 1,
                                                         time_cut=date_eoit.isot)
focus_offset_sum_df = await client.select_top_n("lsst.sal.ATAOS.logevent_focusOffsetSummary", "*", 1,
                                                time_cut=date_start_int.isot)
atmcs_target_df = await client.select_top_n("lsst.sal.ATMCS.logevent_target", "*", 1,
                                            time_cut=date_start_int.isot)
atmcs_m3selected_df = await client.select_top_n("lsst.sal.ATMCS.logevent_m3PortSelected", "selected", 1,
                                                time_cut=date_start_int.isot)
atspec_settings_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_settingsAppliedValues", "*", 1,
                                               time_cut=date_start_int.isot)

if obs_date >= 20210318:
    params = "azimuthCalculatedAngle0,declination0,elevationCalculatedAngle0,ra0,skyAngle0"
    mount_positions_start_df = await client.select_top_n("lsst.sal.ATPtg.mountPositions", params, 1,
                                                        time_cut=date_start_int.isot)
    mount_positions_end_df = await client.select_top_n("lsst.sal.ATPtg.mountPositions", params, 1,
                                                       time_cut=date_eoit.isot)
else:
    mount_positions_start_df = await client.select_top_n("lsst.sal.ATPtg.mount_positions", "*", 1,
                                                        time_cut=date_start_int.isot)
    mount_positions_end_df = await client.select_top_n("lsst.sal.ATPtg.mount_positions", "*", 1,
                                                       time_cut=date_eoit.isot)

In [ ]:
hc.check("OBJECT", header, hu.get_value(current_target_df, "targetName"))
hc.check_float("ROTPA", header, hu.get_value(mount_positions_start_df, "skyAngle0"))
hc.check("ROTCOORD", header, "sky")
if header["IMGTYPE"] in ["OBJECT", "ENGTEST"]:
    hc.check_float("RA", header, np.degrees(hu.get_value(current_target_df, "ra")))
    hc.check_float("DEC", header, np.degrees(hu.get_value(current_target_df, "declination")))
    hc.check_float("RASTART", header, hu.get_value(mount_positions_start_df, "ra0"))
    hc.check_float("RAEND", header, hu.get_value(mount_positions_end_df, "ra0"))
    hc.check_near("RA", "RASTART", header, 0.5)
    hc.check_near("RA", "RAEND", header, 0.5)
    hc.check_float("DECSTART", header, hu.get_value(mount_positions_start_df, "declination0"))
    hc.check_float("DECEND", header, hu.get_value(mount_positions_end_df, "declination0"))
    hc.check_near("DEC", "DECSTART", header, 0.1)
    hc.check_near("DEC", "DECEND", header, 0.1)
    hc.check_float("HASTART", header, hu.get_value(current_target_status_start_df, "ha"))
    hc.check_float("HAEND", header, hu.get_value(current_target_status_end_df, "ha"))
    hc.check_float("AMSTART", header, hu.get_value(current_target_status_start_df, "airmass"))
    hc.check_float("AMEND", header, hu.get_value(current_target_status_end_df, "airmass"))
else:
    print(f"No check on RA/DEC for {header['IMGTYPE']}")
hc.check_float("AZSTART", header, hu.get_value(mount_positions_start_df, "azimuthCalculatedAngle0"))
hc.check_float("AZEND", header, hu.get_value(mount_positions_end_df, "azimuthCalculatedAngle0"))
hc.check_float("ELSTART", header, hu.get_value(mount_positions_start_df, "elevationCalculatedAngle0"))
hc.check_float("ELEND", header, hu.get_value(mount_positions_end_df, "elevationCalculatedAngle0"))
hc.check_float("FOCUSZ", header, hu.get_value(focus_offset_sum_df, "userApplied"))
hc.check("TRACKSYS", header, hu.get_value(atmcs_target_df, "tracksys"))
hc.check("RADESYS", header, hu.get_value(atmcs_target_df, "radesys"))
hc.check("INSTPORT", header, hu.get_value(atspec_settings_df, "instrumentPort"))
hc.check("ATM3PORT", header, hu.get_value(atmcs_m3selected_df, "selected"))

In [ ]:
sim_atmcs_df = await client.select_top_n("lsst.sal.ATMCS.logevent_simulationMode", "mode", 1,
                                         time_cut=date_start_int.isot)
sim_atpneumatics_df = await client.select_top_n("lsst.sal.ATPneumatics.logevent_simulationMode", "mode", 1,
                                                time_cut=date_start_int.isot)
sim_athexapod_df = await client.select_top_n("lsst.sal.ATHexapod.logevent_simulationMode", "mode", 1,
                                             time_cut=date_start_int.isot)
sim_atspectrograph_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_simulationMode", "mode", 1,
                                                  time_cut=date_start_int.isot)
sim_atdome_df = await client.select_top_n("lsst.sal.ATDome.logevent_simulationMode", "mode", 1,
                                          time_cut=date_start_int.isot)

In [ ]:
hc.check("SIMULATE ATMCS", header, hu.get_value(sim_atmcs_df, "mode"))
hc.check("SIMULATE ATPNEUMATICS", header, hu.get_value(sim_atpneumatics_df ,"mode"))
hc.check("SIMULATE ATHEXAPOD", header, hu.get_value(sim_athexapod_df, "mode"))
hc.check("SIMULATE ATSPECTROGRAPH", header, hu.get_value(sim_atspectrograph_df, "mode"))
hc.check("SIMULATE ATDOME", header, hu.get_value(sim_atdome_df, "mode"))

In [ ]:
filter_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedFilterPosition", "*", 1,
                                          time_cut=date_start_int.isot)
grating_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedDisperserPosition", "*", 1,
                                           time_cut=date_start_int.isot)
linear_stage_pos_df = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedLinearStagePosition",
                                                "*", 1,
                                                time_cut=date_start_int.isot)
client.influx_client.output = "json"
filter_pos_json = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedFilterPosition", "*", 1,
                                            time_cut=date_start_int.isot)
grating_pos_json = await client.select_top_n("lsst.sal.ATSpectrograph.logevent_reportedDisperserPosition", "*", 1,
                                             time_cut=date_start_int.isot)
client.influx_client.output = "dataframe"

In [ ]:
hc.check("FILTER", header, hu.get_from_json("name", filter_pos_json))
hc.check("FILTBAND", header, hu.get_value(filter_pos_df, "band"))
hc.check("FILTSLOT", header, hu.get_value(filter_pos_df, "slot"))
hc.check("GRATING", header, hu.get_from_json("name", grating_pos_json))
hc.check("GRATBAND", header, hu.get_value(grating_pos_df, "band"))
hc.check("GRATSLOT", header, hu.get_value(grating_pos_df, "slot"))
hc.check_float("LINSPOS", header, hu.get_value(linear_stage_pos_df, "position"))

In [ ]:
focal_plane_info_df = await client.select_top_n("lsst.sal.ATCamera.logevent_focalPlaneSummaryInfo", "*", 1,
                                                time_cut=date_start_int.isot)
fp_ccd_df = await client.select_top_n("lsst.sal.ATCamera.focal_plane_Ccd", "*", 1,
                                      time_cut=date_start_int.isot)
vacuum_df = await client.select_top_n("lsst.sal.ATCamera.vacuum", "*", 1,
                                      time_cut=date_start_int.isot)

In [ ]:
hc.check("CCD_MANU", header, hu.get_value(focal_plane_info_df, "ccdManufacturer"))
hc.check("CCD_TYPE", header, hu.get_value(focal_plane_info_df, "ccdType"))
hc.check("CCD_SERN", header, hu.get_value(focal_plane_info_df, "ccdManSerNum"))
hc.check("LSST_NUM", header, hu.get_value(focal_plane_info_df, "ccdLSSTName"))
hc.check("CCDSLOT", header, hu.get_value(focal_plane_info_df, "ccdSlot"))
hc.check("RAFTBAY", header, hu.get_value(focal_plane_info_df, "raftBay"))
hc.check("SEQCKSUM", header, hu.get_value(focal_plane_info_df, "sequencerChecksum"))
hc.check("SEQNAME", header, hu.get_value(focal_plane_info_df, "sequencerKey"))
hc.check("REBNAME", header, hu.get_value(focal_plane_info_df, "rebLSSTName"))
hc.check("CONTNUM", header, hu.get_value(focal_plane_info_df, "rebSerialNumber")) 
hc.check_float("CCDTEMP", header, hu.get_value(fp_ccd_df, "temp0"))
hc.check_float("TEMP_SET", header, hu.get_value(focal_plane_info_df, "ccdTempSetPoint"))
hc.check("IMAGETAG", header, hu.get_value(end_image_tel, "imageTag"))

In [ ]:
weather_df = await client.select_top_n("lsst.sal.WeatherStation.weather", "*", 1,
                                       time_cut=date_start_int.isot)
wind_speed_df = await client.select_top_n("lsst.sal.WeatherStation.windSpeed", "*", 1,
                                          time_cut=date_start_int.isot)
wind_dir_df = await client.select_top_n("lsst.sal.WeatherStation.windDirection", "*", 1,
                                        time_cut=date_start_int.isot)
seeing_df = await client.select_top_n("lsst.sal.DIMM.logevent_dimmMeasurement", "*", 1,
                                      time_cut=date_start_int.isot)
dome_pos_df = await client.select_top_n("lsst.sal.ATDome.position", "*", 1,
                                        time_cut=date_start_int.isot)

In [ ]:
hc.check_float("AIRTEMP", header, hu.get_value(weather_df, "ambient_temp"))
hc.check_float("PRESSURE", header, hu.get_value(weather_df, "pressure"))
hc.check_float("HUMIDITY", header, hu.get_value(weather_df, "humidity"))
hc.check_float("WINDSPD", header, hu.get_value(wind_speed_df, "value"))
hc.check_float("WINDDIR", header, hu.get_value(wind_dir_df, "value"))
hc.check_float("SEEING", header, hu.get_value(seeing_df, "fwhm"))
hc.check_float("DOMEAZ", header, hu.get_value(dome_pos_df, "azimuthPosition"))
hc.check_float("SHUTLOWR", header, hu.get_value(dome_pos_df, "dropoutDoorOpeningPercentage"))
hc.check_float("SHUTUPPR", header, hu.get_value(dome_pos_df, "mainDoorOpeningPercentage"))
hc.check("FACILITY", header, "Vera C. Rubin Observatory")

In [ ]:
hc.check_not_empty("DATE", header)
hc.check_not_empty("MJD-OBS", header)
hc.check_not_empty("MJD-BEG", header)
hc.check_not_empty("MJD-END", header)
hc.check("BUNIT", header, "adu")
hc.check("TELESCOP", header, "LSST AuxTelescope")
hc.check("INSTRUME", header, "LATISS")
hc.check_float("OBS-LONG", header, -70.749417)
hc.check_float("OBS-LAT", header, -30.244639)
hc.check_float("OBS-ELEV", header, 2663.0)
hc.check_float("OBSGEO-X", header, 1818938.94)
hc.check_float("OBSGEO-Y", header, -5208470.95)
hc.check_float("OBSGEO-Z", header, -3195172.08)
hc.check_not_empty("FILENAME", header)
hc.check_not_empty("HEADVER", header)
hc.check_not_empty("TSTAND", header)
hc.check_not_empty("CHECKSUM", header)
hc.check_not_empty("DATASUM", header)
hc.check("TIMESYS", header, "TAI")

In [ ]:
hc.check("XTENSION", header, "IMAGE")
hc.check("BITPIX", header, 32)
hc.check("INHERIT", header, True)
hc.check("PCOUNT", header, 0)
hc.check("GCOUNT", header, 1)
hc.check_not_empty("EXTNAME", header)
hc.check_not_empty("DATASEC", header)
hc.check_not_empty("DETSEC", header)
hc.check_not_empty("DTV1", header)
hc.check_not_empty("DTV2", header)
hc.check_not_empty("DTM1_1", header)
hc.check_not_empty("DTM1_2", header)
hc.check_not_empty("DTM2_1", header)
hc.check_not_empty("DTM2_2", header)